In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from icap.database.icapdatabase import ICapDatabase
from icap.peco.peco import PECO
from icap.peco.peco import PECOInterval
from icap.peco.peco import PECOConsumption
from icap.peco.peco import PECODemand

from icap.results.results import Results

from icap.peco.peco import PECORecipe

In [2]:
fp ='icap/database/icapdatabase.json'
conn = ICapDatabase(fp).connect()

In [3]:
res = PECORecipe(conn, Results).run_all()

In [4]:
r = res.analyze_comparison()

In [5]:
r

Passing                          False  True  NULL    All
MeterType RateClass Strata Year                          
CON       OP        110    2014      0     0     1      1
          R         111    2014      0     3     1      4
                           2015      0    10     6     16
                           2016      0    19     2     21
                           2017      0     0    17     17
                    112    2014      2     1     0      3
                           2015      1     4     1      6
                           2016      4     2     0      6
                           2017      0     0     2      2
                    113    2014      1     1     0      2
                           2015      1     4     0      5
                           2016      0    21     0     21
                           2017      0     0    20     20
          RH        121    2014      0     0     1      1
                    122    2015      1     0     0      1
                           2016      1     0     0      1
                    123    2014      1     0     0      1
                           2015      4     0     0      4
                           2016     10     0     0     10
                           2017      0     0     9      9
DMD       GS        101    2015     34     1   130    165
                           2016    249    47   368    664
                           2017      0     0   213    213
                    107    2016      7     8    15     30
                           2017      0     0    12     12
          HT        151    2015      0     0     1      1
                           2016     11     1    15     27
                           2017      0     0     7      7
          PD        157    2015      0     0     2      2
                           2016      7     4     6     17
                           2017      0     0     5      5
INT       GS        101    2015     22     7  1381   1410
                           2016    127  1087  3079   4293
                           2017      0     0   692    692
                    107    2015     43     0   730    773
                           2016    132     2   770    904
                           2017      0     0   108    108
          HT        151    2015      3     8   362    373
                           2016      0    77   327    404
                           2017      0     0    57     57
          PD        157    2015      0     0    11     11
                           2016      0    24    31     55
                           2017      0     0     9      9
          R         111    2015      0     0     1      1
                           2016      0     0     1      1
                           2017      0     0     8      8
                    112    2017      0     0     6      6
                    113    2016      1     0     0      1
                           2017      0     0     9      9
          RH        121    2017      0     0     2      2
                    123    2017      0     0     1      1
All                                662  1331  8419  10412

In [ ]:
r[r['HistVar'] <= 2.0].shape

In [ ]:
r.columns

In [ ]:
r.dropna(axis=0, subset=['CapacityTagValue']).shape

In [ ]:
def passing_value(series):
    variance = series['HistVar']
    if variance <= 2.0:
        return True
    elif variance > 2.0:
        return False
    else:
        return 'NULL'
        
r['Passing'] = r.apply(passing_value, axis=1)

In [ ]:
out = pd.crosstab([r['RateClass'], r['Strata']], r['Passing'], margins=True)
out['PercentPassing'] = out[True] / out['All']

In [ ]:
out_yr = pd.crosstab([r['MeterType'], r['RateClass'], r['Strata'], r['Year']], r['Passing'],margins=True)
# out_yr['PercentPassing'] = out_yr[True] / out['All']

In [ ]:
out_yr

In [ ]:
res = res.compare_.copy()

In [ ]:
valid_idx = res[res['HistVar'] <= 2.0].index

res['Valid'] = 0
res.set_value(valid_idx, 'Valid', 1)

res.dropna(subset=['HistVar'], inplace=True)
count = res.groupby('Valid')['Valid'].count()

## PECO super class

In [ ]:
p = PECO(conn)

In [ ]:
# confirm utility/system/loadshape dataframes are NOT empty
all((not p.util_df_.empty, not p.sys_df_.empty, not p.loadshape_df_.empty))

## PECODemand

In [ ]:
pdmd_case = PECODemand(conn, premise='3612801108')

## PECOConsumption

In [ ]:
pcon = PECOConsumption(conn, premise='5811500607').compute_icap()

In [ ]:
pcon

## PECOInterval

In [ ]:
pint = PECOInterval(conn, premise='8722801404')

In [ ]:
pint.records_[pint.records_['Year'] == '2014'
             ].sort_values(by='UsageDate')['Usage']

## Concatenate and Compute Historical

In [ ]:
av = pd.concat([pdmd, pcon, pint])
Results(conn, av).write_to_csv()
print('Done!')

In [ ]:
df = pd.read_csv('peco_rec.csv')

In [ ]:
df.head()

In [ ]:
df.dropna(subset=['HistVar'], inplace=True)
df['Valid'] = False
valid_idx = df[df['HistVar'] <= 2.0].index
df.set_value(valid_idx, 'Valid', True)


by_year = df.groupby(
    ['Year', 'RateClass', 'Strata', 'Valid'])['HistVar'].count().unstack(level=3
                                                                        ).replace(to_replace=np.nan, value=0.0
                                                                                 )

In [ ]:
by_year['Ratio'] = by_year[True] / (by_year[True] + by_year[False])

In [ ]:
def avg(grp, label=None):
    grp[label] = grp['Ratio'].mean()
    return grp

t_1 = by_year.reset_index().groupby(['Year', 'RateClass']).apply(lambda x: avg(x, 'RC_AVG'))
#t_2 = by_year.reset_index().groupby(['Year']).apply(lambda x: avg(x, 'Year_AVG'))

t_1 = t_1.groupby('Year').apply(lambda x: avg(x, 'Year_AVG'));
t_1['Count']

In [ ]:
writer = pd.ExcelWriter('/home/miles/Desktop/peco_historical_comparison.xlsx', engine='xlsxwriter')
tmp.to_excel(writer, sheet_name='Analysis')
writer.save()

In [ ]:
pd.read_excel('/home/miles/Downloads/peco_historical_comparison.xlsx')